### Twitter Text

In [ ]:
import twitter

# Twitter API keys go here
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''




auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [ ]:
q = '#FirstWorldProblems' 
count = 100

tweet_texts = []
tweet_mentions = []
tweet_hashtags = []
tweet_bios = []
ids = []

search_results = twitter_api.search.tweets(q=q, count=count)
statuses = search_results['statuses']

# save relevant data
ids += [status['id'] for status in statuses]
tweet_texts += [status['text'] for status in statuses]
tweet_mentions += [mention['screen_name'] for status in statuses
                       for mention in status['entities']['user_mentions']]
tweet_hashtags += [hashtag['text'] for status in statuses
                       for hashtag in status['entities']['hashtags']]
tweet_bios += [status['user']['description'] for status in statuses 
                       if status['user']['description']]

In [ ]:
# use a loop
num_iterations = 20
print len(set(ids))
for i in range(num_iterations):
    params = {a:b for a,b in [x.split('=') for x in search_results['search_metadata']['next_results'][1:].split('&')]}
    max_id = int(params['max_id'])
    search_results = twitter_api.search.tweets(q=q, count=count, max_id=max_id)
    statuses = search_results['statuses']
    
    # save relevant data
    tweet_texts += [status['text'] for status in statuses]
    tweet_mentions += [mention['screen_name'] for status in statuses
                           for mention in status['entities']['user_mentions']]
    tweet_hashtags += [hashtag['text'] for status in statuses
                           for hashtag in status['entities']['hashtags']]
    tweet_bios += [status['user']['description'] for status in statuses 
                           if status['user']['description']]
    
    ids += [status['id'] for status in statuses]
    print len(set(ids))

In [ ]:
# how many more calls do we have?
twitter_api.application.rate_limit_status()['resources']['search']['/search/tweets']

In [ ]:
len(tweet_texts)

In [ ]:
# only represents a few different tweets
len(set(tweet_texts))

In [ ]:
from collections import Counter

Counter(tweet_texts).most_common()[:10]

In [ ]:
import json
# Compute a collection of all words from all tweets
words = [w for tweet_text in tweet_texts for w in tweet_text.split()]

print json.dumps(words[0:10], indent=1)

In [ ]:
bio_words = [w for bio in tweet_bios for w in bio.split()]

In [ ]:
# print top items from each array
from collections import Counter

for item in [words, bio_words, tweet_mentions, tweet_hashtags]:
    c = Counter(item)
    print c.most_common()[:10] # top 10
    print ''

### Clean up our text

In [ ]:
# Regular Expression
import re

stub = re.compile('[^A-Za-z]')

In [ ]:
c = Counter(tweet_texts)

for tw, v in c.most_common(2):
    print tw

In [ ]:
# length <=2 - unimportant?
for tw, v in c.most_common(2):
    print ' '.join([stub.sub('',w) for w in tw.split()])

In [ ]:
# get rid of words that start with '@'

for tw, v in c.most_common(2):
    print ' '.join([stub.sub('',w) for w in tw.split() if not w.startswith('@')])

### Common Words

In [ ]:
blacklist = ['i','am','for','can','via','with','top','las','una','que','hay','todo','por','the','going','you','are','not','its','until']

for tw, v in c.most_common(10):
    cur_tweet = []
    for w in tw.split():
        if w.startswith('@'):
            continue
        cur_w = stub.sub('', w)
        if len(cur_w)<3:
            continue
        if cur_w.lower() in blacklist:
            continue
            
        cur_tweet.append(cur_w)
        
    print ' '.join(cur_tweet)

In [ ]:
# pip install nltk
import nltk
nltk.download()

In [ ]:
# To get this working I've had to restart my ipython notebook server (which is a bit annoying -> 
# but we can do it together in class)
# easiest is to simply restart our machines

# Here's the command: 
#          /usr/bin/python /usr/local/bin/ipython notebook --notebook-dir=/class --no-browser --port=8887 &


from nltk.corpus import stopwords

stopwords.words('english')

In [ ]:
for tw, v in c.most_common(10):
    for w in tw.split():
        if w.startswith('@'):
            continue
        cur_w = stub.sub('', w)
        if len(cur_w)<3:
            continue
        if cur_w.lower() in stopwords.words('english'):
            continue
        if cur_w.lower() in stopwords.words('spanish'):
            continue
            
        print cur_w

In [ ]:
# common words
common_words = ['yes','way','youre']

for tw, v in c.most_common(10):
    for w in tw.split():
        if w.startswith('http://'):
            continue
        cur_w = stub.sub('', w)
        if len(cur_w)<3:
            continue
        if cur_w.lower() in stopwords.words('english'):
            continue
        if cur_w.lower() in stopwords.words('spanish'):
            continue
        if cur_w.lower() in common_words:
            continue
            
        print cur_w


### Find Hashtags, Mentions using a regular expression

In [ ]:
# Hashtags
import re

re_hash = re.compile(r'#[0-9a-zA-Z+_]*',re.IGNORECASE)
hashtags = Counter()

for tw in c.keys():
    hashtags.update(re.findall(re_hash, tw))

# ---> other ways to do this:
# print re.findall(r'#(\w+)', tw)
# print re.findall(r'\B#\w+', tw)

In [ ]:
hashtags.most_common(10)

In [ ]:
# Mentions
re_mention = re.compile(r'@[0-9a-zA-Z+_]*',re.IGNORECASE)
mentions = Counter()

for tw in c.keys():
    mentions.update(re.findall(re_mention, tw))

In [ ]:
mentions.most_common(10)

### Find Capitalized Phrases using a regular expression

In [ ]:
from collections import Counter

phrases = Counter()

comp = re.compile(r'([A-Z][a-z]+(?=\s[A-Z])(?:\s[A-Z][a-z]+)+)')
for tw, v in c.items():
    phrases.update(re.findall(comp, tw))

In [ ]:
phrases.most_common(10)

In [ ]:
len(phrases)

### make a bi-partite graph using top phrases, hashtags and mentions

In [ ]:
limit = 5000

top_phrases = [x for x,y in phrases.most_common(limit)]
top_hashtags = [x for x,y in hashtags.most_common(limit) if len(x)>2]
top_mentions = [x for x,y in mentions.most_common(limit) if len(x)>2]

In [ ]:
top_hashtags[:10]

In [ ]:
top_mentions[:10]

### Generate Graph

In [ ]:
import networkx as nx

# functions that help us construct the graph
def graph_add_node(n, g, node_type=None):
    try:
        if g.has_node(n):
            g.node[n]['weight']+=1
        else:
            g.add_node(n)
            g.node[n]['label'] = n
            g.node[n]['weight'] = 1
            g.node[n]['type']=node_type
    except:
        return
            
def graph_add_edge(n1, n2, g):
    if g.has_edge(n1, n2):
        g[n1][n2]['weight']+=1
    else:
        g.add_edge(n1,n2)
        g[n1][n2]['weight']=1

In [ ]:
# Get item co-occurrence
from itertools import combinations

stub = re.compile('[^A-Za-z#@]')
re_phrase = re.compile(r'([A-Z][a-z]+(?=\s[A-Z])(?:\s[A-Z][a-z]+)+)')
re_mention_hashtag = re.compile(r'[@|#][0-9a-zA-Z+_]*',re.IGNORECASE)

g = nx.Graph()

for t,v in c.most_common(30000):
    cur_items = set()
    
    for phrase in re.findall(re_phrase, t):
        cur_items.add(phrase)
        graph_add_node(phrase, g, node_type='phrase')
        
    for men_hash in re.findall(re_mention_hashtag, t):
        cur_items.add(men_hash)
        if men_hash.startswith('#'):
            graph_add_node(men_hash, g, node_type='hashtag')
        elif men_hash.startswith('@'):
            graph_add_node(men_hash, g, node_type='mention')
        
    # add edges
    for i1, i2 in combinations(cur_items, 2):
        graph_add_edge(i1, i2, g)

In [ ]:
print g.number_of_nodes()
print g.number_of_edges()

In [ ]:
# output file
q = 'firstworldproblems' 
path = '/class/itpmssd/datasets/'
nx.write_gexf(g, path+'%s_tweet_graph.gexf' % q)
print path+'%s_tweet_graph.gexf' % q

## Hashtags with Highest Betweenness Centrality

In [ ]:
components = sorted(nx.connected_component_subgraphs(g), key=len, reverse=True)
len(components[0])

In [ ]:
betweenness = nx.betweenness_centrality(components[0])

In [ ]:
for k,v in sorted(betweenness.items(), key=lambda x:-x[1])[:10]:
    print k,v